## **Setup**

In [1]:
from google.colab import drive
mount_path = '/content/gdrive/'
drive.mount(mount_path)
competition_path = 'My Drive/AI For Good - AI Blitz 3/Snake/Data/'
results_path='My Drive/AI For Good - AI Blitz 3/Snake/Results/'
model_path='My Drive/AI For Good - AI Blitz 3/Snake/Models/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/4AEo_YUjsib3KHFh63bYu5zrnDTzvpfqUWI683V9OoMmXYgl55pWAck
Mounted at /content/gdrive/


In [2]:
import os
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import pytz
from zipfile import ZipFile
from tempfile import TemporaryDirectory
import requests
import cv2
import shutil
import glob

# from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.model_selection import train_test_split

# plot options
# plt.rcParams.update({'font.size': 11})
plt.style.use('fivethirtyeight')

# Data Loading

### Training Data

In [59]:
preds_files=glob.glob(f'{mount_path}{results_path}*non_venomous.npy')
print(len(preds_files))
preds_files

18


['/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Results/200830_0917_resnet34_cv0_non_venomous.npy',
 '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Results/200830_1038_resnet34_cv1_non_venomous.npy',
 '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Results/200830_1158_resnet34_cv2_non_venomous.npy',
 '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Results/200830_1321_xresnet34_cv0_non_venomous.npy',
 '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Results/200830_1443_xresnet34_cv1_non_venomous.npy',
 '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Results/200830_1605_xresnet34_cv2_non_venomous.npy',
 '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Results/200830_1734_resnet50_cv0_non_venomous.npy',
 '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Results/200830_1904_resnet50_cv1_non_venomous.npy',
 '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Results/200830_2035_resnet50_cv2_non_venomous.npy',
 '/cont

 Organize the predcitions into a DataFrame

In [60]:
all_preds=[]
col_names=[]
for preds_file in preds_files:
  curpreds=np.concatenate((np.load(preds_file),np.load(preds_file.replace('_non_','_'))),0)

  all_preds.append(curpreds[:,0])
  all_preds.append(curpreds[:,1])

  modelname=os.path.basename(preds_file)
  modelname=modelname[:(modelname.index('_cv')+4)]

  col_names.append(modelname+'_0')
  col_names.append(modelname+'_1')

pred_df=pd.DataFrame(data=np.transpose(np.array(all_preds)),columns=col_names)

In [61]:
pred_df.corr()

,200830_0917_resnet34_cv0_0,200830_0917_resnet34_cv0_1,200830_1038_resnet34_cv1_0,200830_1038_resnet34_cv1_1,200830_1158_resnet34_cv2_0,200830_1158_resnet34_cv2_1,200830_1321_xresnet34_cv0_0,200830_1321_xresnet34_cv0_1,200830_1443_xresnet34_cv1_0,200830_1443_xresnet34_cv1_1,200830_1605_xresnet34_cv2_0,200830_1605_xresnet34_cv2_1,200830_1734_resnet50_cv0_0,200830_1734_resnet50_cv0_1,200830_1904_resnet50_cv1_0,200830_1904_resnet50_cv1_1,200830_2035_resnet50_cv2_0,200830_2035_resnet50_cv2_1,200831_0244_densenet121_cv0_0,200831_0244_densenet121_cv0_1,200831_0726_densenet201_cv0_0,200831_0726_densenet201_cv0_1,200831_0410_densenet121_cv1_0,200831_0410_densenet121_cv1_1,200831_0917_densenet201_cv1_0,200831_0917_densenet201_cv1_1,200831_0535_densenet121_cv2_0,200831_0535_densenet121_cv2_1,200831_1109_densenet201_cv2_0,200831_1109_densenet201_cv2_1,200830_2227_xresnet50_cv0_0,200830_2227_xresnet50_cv0_1,200830_2352_xresnet50_cv1_0,200830_2352_xresnet50_cv1_1,200831_0116_xresnet50_cv2_0,200831_0116_xresnet50_cv2_1
200830_0917_resnet34_cv0_0,1.000000,-1.000000,0.958208,-0.958208,0.948926,-0.948926,0.809538,-0.809538,0.806538,-0.806538,0.806418,-0.806418,0.927768,-0.927768,0.661185,-0.661185,0.664254,-0.664254,0.692881,-0.692881,0.702585,-0.702585,0.695418,-0.695418,0.698344,-0.698344,0.694459,-0.694459,0.703201,-0.703201,0.543862,-0.543862,0.550900,-0.550900,0.548762,-0.548762
200830_0917_resnet34_cv0_1,-1.000000,1.000000,-0.958208,0.958208,-0.948926,0.948926,-0.809538,0.809538,-0.806538,0.806538,-0.806418,0.806418,-0.927768,0.927768,-0.661185,0.661185,-0.664254,0.664254,-0.692881,0.692881,-0.702585,0.702585,-0.695418,0.695418,-0.698344,0.698344,-0.694459,0.694459,-0.703201,0.703201,-0.543862,0.543862,-0.550900,0.550900,-0.548762,0.548762
200830_1038_resnet34_cv1_0,0.958208,-0.958208,1.000000,-1.000000,0.949839,-0.949839,0.810450,-0.810450,0.807977,-0.807977,0.806175,-0.806175,0.928461,-0.928461,0.660356,-0.660356,0.664110,-0.664110,0.692357,-0.692357,0.702267,-0.702267,0.694732,-0.694732,0.697678,-0.697678,0.694166,-0.694166,0.701999,-0.701999,0.544210,-0.544210,0.551416,-0.551416,0.548888,-0.548888
200830_1038_resnet34_cv1_1,-0.958208,0.958208,-1.000000,1.000000,-0.949839,0.949839,-0.810450,0.810450,-0.807977,0.807977,-0.806175,0.806175,-0.928461,0.928461,-0.660356,0.660356,-0.664110,0.664110,-0.692357,0.692357,-0.702267,0.702267,-0.694732,0.694732,-0.697678,0.697678,-0.694166,0.694166,-0.701999,0.701999,-0.544210,0.544210,-0.551416,0.551416,-0.548888,0.548888
200830_1158_resnet34_cv2_0,0.948926,-0.948926,0.949839,-0.949839,1.000000,-1.000000,0.792616,-0.792616,0.790184,-0.790184,0.788994,-0.788994,0.920909,-0.920909,0.662529,-0.662529,0.665658,-0.665658,0.694230,-0.694230,0.704354,-0.704354,0.696552,-0.696552,0.700082,-0.700082,0.696118,-0.696118,0.704322,-0.704322,0.545031,-0.545031,0.552672,-0.552672,0.550013,-0.550013
200830_1158_resnet34_cv2_1,-0.948926,0.948926,-0.949839,0.949839,-1.000000,1.000000,-0.792616,0.792616,-0.790184,0.790184,-0.788994,0.788994,-0.920909,0.920909,-0.662529,0.662529,-0.665658,0.665658,-0.694230,0.694230,-0.704354,0.704354,-0.696552,0.696552,-0.700082,0.700082,-0.696118,0.696118,-0.704322,0.704322,-0.545031,0.545031,-0.552672,0.552672,-0.550013,0.550013
200830_1321_xresnet34_cv0_0,0.809538,-0.809538,0.810450,-0.810450,0.792616,-0.792616,1.000000,-1.000000,0.944287,-0.944287,0.942691,-0.942691,0.784970,-0.784970,0.541263,-0.541263,0.543198,-0.543198,0.566847,-0.566847,0.575528,-0.575528,0.568785,-0.568785,0.572005,-0.572005,0.568420,-0.568420,0.575091,-0.575091,0.445985,-0.445985,0.450313,-0.450313,0.449395,-0.449395
200830_1321_xresnet34_cv0_1,-0.809538,0.809538,-0.810450,0.810450,-0.792616,0.792616,-1.000000,1.000000,-0.944287,0.944287,-0.942691,0.942691,-0.784970,0.784970,-0.541263,0.541263,-0.543198,0.543198,-0.566847,0.566847,-0.575528,0.575528,-0.568785,0.568785,-0.572005,0.572005,-0.568420,0.568420,-0.575091,0.575091,-0.445985,0.445985,-0.450313,0.450313,-0.449395,0.449395
200830_1443_xresnet34_cv1

 Make the label vector

In [62]:
label_nv=np.load(preds_file)[:,0].astype(str)
label_nv[:]='non_venomous'

label_v=np.load(preds_file.replace('_non_','_'))[:,0].astype(str)
label_v[:]='venomous'

labels=np.concatenate((label_nv,label_v)).tolist()

In [63]:
binlabels=np.ones((len(labels),))
binlabels[np.array(labels)=='venomous']=0

In [64]:
np.mean(binlabels)

0.5556709666610823

In [65]:
print(pred_df.shape)
pred_df.head()

(59690, 36)


,200830_0917_resnet34_cv0_0,200830_0917_resnet34_cv0_1,200830_1038_resnet34_cv1_0,200830_1038_resnet34_cv1_1,200830_1158_resnet34_cv2_0,200830_1158_resnet34_cv2_1,200830_1321_xresnet34_cv0_0,200830_1321_xresnet34_cv0_1,200830_1443_xresnet34_cv1_0,200830_1443_xresnet34_cv1_1,200830_1605_xresnet34_cv2_0,200830_1605_xresnet34_cv2_1,200830_1734_resnet50_cv0_0,200830_1734_resnet50_cv0_1,200830_1904_resnet50_cv1_0,200830_1904_resnet50_cv1_1,200830_2035_resnet50_cv2_0,200830_2035_resnet50_cv2_1,200831_0244_densenet121_cv0_0,200831_0244_densenet121_cv0_1,200831_0726_densenet201_cv0_0,200831_0726_densenet201_cv0_1,200831_0410_densenet121_cv1_0,200831_0410_densenet121_cv1_1,200831_0917_densenet201_cv1_0,200831_0917_densenet201_cv1_1,200831_0535_densenet121_cv2_0,200831_0535_densenet121_cv2_1,200831_1109_densenet201_cv2_0,200831_1109_densenet201_cv2_1,200830_2227_xresnet50_cv0_0,200830_2227_xresnet50_cv0_1,200830_2352_xresnet50_cv1_0,200830_2352_xresnet50_cv1_1,200831_0116_xresnet50_cv2_0,200831_0116_xresnet50_cv2_1
0,0.780205,0.219795,0.782473,0.217527,0.836068,0.163932,0.821801,0.178199,0.747963,0.252037,0.707402,0.292598,0.928889,0.071111,0.907861,0.092139,0.922420,0.077580,0.839366,0.160634,0.897784,0.102216,0.765340,0.234660,0.797143,0.202857,0.908533,0.091467,0.827549,0.172451,0.888911,0.111089,0.642826,0.357174,0.895406,0.104594
1,0.983987,0.016013,0.968622,0.031378,0.967417,0.032583,0.645694,0.354306,0.819075,0.180925,0.808127,0.191873,0.985995,0.014005,0.992686,0.007314,0.997364,0.002636,0.994923,0.005077,0.999179,0.000821,0.985221,0.014779,0.998922,0.001078,0.992474,0.007526,0.997985,0.002015,0.538927,0.461073,0.616410,0.383590,0.695873,0.304127
2,0.367011,0.632989,0.697410,0.302590,0.504666,0.495334,0.543355,0.456645,0.595894,0.404106,0.669215,0.330785,0.484954,0.515046,0.976632,0.023368,0.981466,0.018534,0.968495,0.031505,0.997529,0.002471,0.995974,0.004026,0.945479,0.054521,0.537240,0.462760,0.971950,0.028050,0.692725,0.307275,0.535327,0.464673,0.562500,0.437500
3,0.843469,0.156531,0.835188,0.164812,0.814259,0.185741,0.675851,0.324149,0.742603,0.257397,0.724972,0.275028,0.883421,0.116579,0.969162,0.030838,0.931411,0.068589,0.991098,0.008902,0.989626,0.010374,0.983859,0.016141,0.987888,0.012112,0.975484,0.024517,0.975781,0.024219,0.948102,0.051898,0.924924,0.075076,0.977763,0.022237
4,0.916911,0.083089,0.985165,0.014835,0.859182,0.140818,0.592059,0.407941,0.659934,0.340066,0.662165,0.337835,0.946330,0.053670,0.960941,0.039059,0.959137,0.040863,0.997939,0.002061,0.987190,0.012810,0.996019,0.003981,0.987513,0.012487,0.997095,0.002905,0.996957,0.003043,0.662194,0.337806,0.726540,0.273460,0.691945,0.308056


### Test Data

In [66]:
testpreds_files=glob.glob(f'{mount_path}{results_path}*test.npy')
print(len(testpreds_files))
testpreds_files

18


['/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Results/200831_0726_densenet201_cv0_test.npy',
 '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Results/200830_0917_resnet34_cv0_test.npy',
 '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Results/200830_1605_xresnet34_cv2_test.npy',
 '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Results/200831_0244_densenet121_cv0_test.npy',
 '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Results/200830_1038_resnet34_cv1_test.npy',
 '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Results/200830_1734_resnet50_cv0_test.npy',
 '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Results/200830_1158_resnet34_cv2_test.npy',
 '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Results/200831_1109_densenet201_cv2_test.npy',
 '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Results/200831_0116_xresnet50_cv2_test.npy',
 '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Results/20083

In [67]:
all_preds=[]
col_names=[]
for preds_file in testpreds_files:
  curpreds=np.load(preds_file)

  print(f'{np.mean(np.argmax(curpreds,axis=1)):.3f}')
  # print(curpreds.shape)

  all_preds.append(curpreds[:,0])
  all_preds.append(curpreds[:,1])

  modelname=os.path.basename(preds_file)
  modelname=modelname[:(modelname.index('_cv')+4)]

  col_names.append(modelname+'_0')
  col_names.append(modelname+'_1')

testpred_df=pd.DataFrame(data=np.transpose(np.array(all_preds)),columns=col_names)

0.417
0.414
0.363
0.407
0.418
0.414
0.410
0.430
0.371
0.386
0.415
0.370
0.411
0.415
0.398
0.420
0.355
0.422


In [68]:
testpred_df=testpred_df[pred_df.columns]

print(testpred_df.shape)
testpred_df.head()

(14927, 36)


,200830_0917_resnet34_cv0_0,200830_0917_resnet34_cv0_1,200830_1038_resnet34_cv1_0,200830_1038_resnet34_cv1_1,200830_1158_resnet34_cv2_0,200830_1158_resnet34_cv2_1,200830_1321_xresnet34_cv0_0,200830_1321_xresnet34_cv0_1,200830_1443_xresnet34_cv1_0,200830_1443_xresnet34_cv1_1,200830_1605_xresnet34_cv2_0,200830_1605_xresnet34_cv2_1,200830_1734_resnet50_cv0_0,200830_1734_resnet50_cv0_1,200830_1904_resnet50_cv1_0,200830_1904_resnet50_cv1_1,200830_2035_resnet50_cv2_0,200830_2035_resnet50_cv2_1,200831_0244_densenet121_cv0_0,200831_0244_densenet121_cv0_1,200831_0726_densenet201_cv0_0,200831_0726_densenet201_cv0_1,200831_0410_densenet121_cv1_0,200831_0410_densenet121_cv1_1,200831_0917_densenet201_cv1_0,200831_0917_densenet201_cv1_1,200831_0535_densenet121_cv2_0,200831_0535_densenet121_cv2_1,200831_1109_densenet201_cv2_0,200831_1109_densenet201_cv2_1,200830_2227_xresnet50_cv0_0,200830_2227_xresnet50_cv0_1,200830_2352_xresnet50_cv1_0,200830_2352_xresnet50_cv1_1,200831_0116_xresnet50_cv2_0,200831_0116_xresnet50_cv2_1
0,0.998952,0.001048,0.994257,0.005743,0.995571,0.004429,0.735410,0.264590,0.772338,0.227662,0.723867,0.276133,0.991011,0.008989,0.998813,0.001187,0.968105,0.031895,0.988026,0.011974,0.994953,0.005047,0.996417,0.003583,0.995349,0.004651,0.998972,0.001028,0.999045,0.000955,0.754077,0.245923,0.815132,0.184868,0.809923,0.190077
1,0.414366,0.585634,0.554772,0.445228,0.323922,0.676078,0.599652,0.400348,0.493560,0.506440,0.793262,0.206738,0.592157,0.407843,0.757156,0.242844,0.675560,0.324440,0.736438,0.263562,0.643716,0.356284,0.847452,0.152548,0.820010,0.179990,0.857214,0.142786,0.587408,0.412592,0.692791,0.307209,0.743628,0.256372,0.660726,0.339274
2,0.003731,0.996269,0.001287,0.998713,0.005545,0.994455,0.012213,0.987787,0.046716,0.953284,0.049666,0.950334,0.000203,0.999797,0.000695,0.999305,0.000551,0.999449,0.001657,0.998343,0.000041,0.999959,0.001066,0.998934,0.000417,0.999583,0.000138,0.999862,0.000566,0.999434,0.021354,0.978646,0.014862,0.985138,0.028814,0.971186
3,0.998891,0.001109,0.999286,0.000714,0.999366,0.000634,0.957096,0.042904,0.989737,0.010263,0.976949,0.023051,0.999730,0.000270,0.999663,0.000337,0.995904,0.004096,0.999925,0.000075,0.999912,0.000088,0.999876,0.000124,0.999986,0.000014,0.999913,0.000087,0.999857,0.000143,0.985604,0.014396,0.982102,0.017898,0.981977,0.018023
4,0.263864,0.736136,0.042251,0.957749,0.108700,0.891300,0.226262,0.773738,0.248583,0.751417,0.218455,0.781545,0.119213,0.880787,0.178948,0.821052,0.193457,0.806543,0.090053,0.909947,0.244046,0.755954,0.019425,0.980575,0.083980,0.916020,0.216218,0.783782,0.066212,0.933788,0.155565,0.844435,0.185898,0.814102,0.197949,0.802051


In [69]:
testpred_df.corr()

,200830_0917_resnet34_cv0_0,200830_0917_resnet34_cv0_1,200830_1038_resnet34_cv1_0,200830_1038_resnet34_cv1_1,200830_1158_resnet34_cv2_0,200830_1158_resnet34_cv2_1,200830_1321_xresnet34_cv0_0,200830_1321_xresnet34_cv0_1,200830_1443_xresnet34_cv1_0,200830_1443_xresnet34_cv1_1,200830_1605_xresnet34_cv2_0,200830_1605_xresnet34_cv2_1,200830_1734_resnet50_cv0_0,200830_1734_resnet50_cv0_1,200830_1904_resnet50_cv1_0,200830_1904_resnet50_cv1_1,200830_2035_resnet50_cv2_0,200830_2035_resnet50_cv2_1,200831_0244_densenet121_cv0_0,200831_0244_densenet121_cv0_1,200831_0726_densenet201_cv0_0,200831_0726_densenet201_cv0_1,200831_0410_densenet121_cv1_0,200831_0410_densenet121_cv1_1,200831_0917_densenet201_cv1_0,200831_0917_densenet201_cv1_1,200831_0535_densenet121_cv2_0,200831_0535_densenet121_cv2_1,200831_1109_densenet201_cv2_0,200831_1109_densenet201_cv2_1,200830_2227_xresnet50_cv0_0,200830_2227_xresnet50_cv0_1,200830_2352_xresnet50_cv1_0,200830_2352_xresnet50_cv1_1,200831_0116_xresnet50_cv2_0,200831_0116_xresnet50_cv2_1
200830_0917_resnet34_cv0_0,1.000000,-1.000000,0.957047,-0.957047,0.956446,-0.956446,0.801897,-0.801897,0.799913,-0.799913,0.799895,-0.799895,0.935838,-0.935838,0.937316,-0.937316,0.939998,-0.939998,0.931406,-0.931406,0.922505,-0.922505,0.929513,-0.929513,0.925827,-0.925827,0.927432,-0.927432,0.922603,-0.922603,0.808269,-0.808269,0.815859,-0.815859,0.816348,-0.816348
200830_0917_resnet34_cv0_1,-1.000000,1.000000,-0.957047,0.957047,-0.956446,0.956446,-0.801897,0.801897,-0.799913,0.799913,-0.799895,0.799895,-0.935838,0.935838,-0.937316,0.937316,-0.939998,0.939998,-0.931406,0.931406,-0.922505,0.922505,-0.929513,0.929513,-0.925827,0.925827,-0.927432,0.927432,-0.922603,0.922603,-0.808269,0.808269,-0.815859,0.815859,-0.816348,0.816348
200830_1038_resnet34_cv1_0,0.957047,-0.957047,1.000000,-1.000000,0.957566,-0.957566,0.805797,-0.805797,0.803439,-0.803439,0.803374,-0.803374,0.933636,-0.933636,0.934661,-0.934661,0.938394,-0.938394,0.926738,-0.926738,0.918057,-0.918057,0.926519,-0.926519,0.922798,-0.922798,0.924560,-0.924560,0.919184,-0.919184,0.810900,-0.810900,0.821105,-0.821105,0.818359,-0.818359
200830_1038_resnet34_cv1_1,-0.957047,0.957047,-1.000000,1.000000,-0.957566,0.957566,-0.805797,0.805797,-0.803439,0.803439,-0.803374,0.803374,-0.933636,0.933636,-0.934661,0.934661,-0.938394,0.938394,-0.926738,0.926738,-0.918057,0.918057,-0.926519,0.926519,-0.922798,0.922798,-0.924560,0.924560,-0.919184,0.919184,-0.810900,0.810900,-0.821105,0.821105,-0.818359,0.818359
200830_1158_resnet34_cv2_0,0.956446,-0.956446,0.957566,-0.957566,1.000000,-1.000000,0.804962,-0.804962,0.803612,-0.803612,0.802903,-0.802903,0.933702,-0.933702,0.934596,-0.934596,0.938582,-0.938582,0.925946,-0.925946,0.917787,-0.917787,0.926359,-0.926359,0.922280,-0.922280,0.923922,-0.923922,0.918350,-0.918350,0.808901,-0.808901,0.819203,-0.819203,0.816997,-0.816997
200830_1158_resnet34_cv2_1,-0.956446,0.956446,-0.957566,0.957566,-1.000000,1.000000,-0.804962,0.804962,-0.803612,0.803612,-0.802903,0.802903,-0.933702,0.933702,-0.934596,0.934596,-0.938582,0.938582,-0.925946,0.925946,-0.917787,0.917787,-0.926359,0.926359,-0.922280,0.922280,-0.923922,0.923922,-0.918350,0.918350,-0.808901,0.808901,-0.819203,0.819203,-0.816997,0.816997
200830_1321_xresnet34_cv0_0,0.801897,-0.801897,0.805797,-0.805797,0.804962,-0.804962,1.000000,-1.000000,0.950786,-0.950786,0.949050,-0.949050,0.792823,-0.792823,0.795290,-0.795290,0.796679,-0.796679,0.771717,-0.771717,0.757491,-0.757491,0.767247,-0.767247,0.761730,-0.761730,0.763972,-0.763972,0.757980,-0.757980,0.937990,-0.937990,0.938901,-0.938901,0.935548,-0.935548
200830_1321_xresnet34_cv0_1,-0.801897,0.801897,-0.805797,0.805797,-0.804962,0.804962,-1.000000,1.000000,-0.950786,0.950786,-0.949050,0.949050,-0.792823,0.792823,-0.795290,0.795290,-0.796679,0.796679,-0.771717,0.771717,-0.757491,0.757491,-0.767247,0.767247,-0.761730,0.761730,-0.763972,0.763972,-0.757980,0.757980,-0.937990,0.937990,-0.938901,0.938901,-0.935548,0.935548
200830_1443_xresnet34_cv1

In [70]:
assert(np.all(testpred_df.columns==pred_df.columns))
print('DataFrames set up correctly')

DataFrames set up correctly


# Train Boosting classifier

In [17]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

### Grid search for best parameters

In [90]:
grid_values = {'learning_rate':[0.01,0.1,0.5],'max_depth':[1,3,5,10],'n_estimators':[50,100,200,500]}

clf=xgb.XGBClassifier()

grid_clf = GridSearchCV(clf, param_grid = grid_values, scoring='f1', cv=3,
                        verbose=10, n_jobs=-1)
grid_clf.fit(pred_df,binlabels)

print('Grid best score (f1): ', grid_clf.best_score_)
print('Grid best parameter (max. f1): ', grid_clf.best_params_)

Fitting 3 folds for each of 48 candidates, totalling 144 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   10.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   36.2s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 16.3min
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed: 18.7min
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed: 20.8min
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed: 24.9min
[Parallel(n_jobs=-1)]: Done  94 tasks      | elapsed: 33.5min
[Parallel(n_jobs=-1)]: Done 109 tasks      | elapsed: 36.6min
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed: 39.9min
[Parallel(n_jobs=-1)]: Done 141 tasks      | elapsed: 46

Grid best score (f1):  0.9790217737784448
Grid best parameter (max. f1):  {'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 200}


### Fit the model w/ the best parameters

Check on a validation set

In [42]:
from sklearn.metrics import f1_score

def f1_eval(y_pred, dtrain):
    y_true = dtrain.get_label()
    return 'f1_err', f1_score(y_true, np.round(y_pred))

In [71]:
X_train, X_val, y_train, y_val = train_test_split(pred_df, binlabels, test_size=0.2)

In [72]:
xgb_model=xgb.XGBClassifier(learning_rate= 0.1, max_depth= 1, n_estimators= 500)

xgb_model.fit(X=X_train,y=y_train, eval_set=[(X_val,y_val)], eval_metric=f1_eval)

[0]	validation_0-error:0.081421	validation_0-f1_err:0.927517
[1]	validation_0-error:0.081421	validation_0-f1_err:0.927517
[2]	validation_0-error:0.074468	validation_0-f1_err:0.933602
[3]	validation_0-error:0.073714	validation_0-f1_err:0.933983
[4]	validation_0-error:0.059809	validation_0-f1_err:0.946828
[5]	validation_0-error:0.068856	validation_0-f1_err:0.938418
[6]	validation_0-error:0.054532	validation_0-f1_err:0.951327
[7]	validation_0-error:0.06006	validation_0-f1_err:0.946441
[8]	validation_0-error:0.052103	validation_0-f1_err:0.953568
[9]	validation_0-error:0.054113	validation_0-f1_err:0.951741
[10]	validation_0-error:0.049422	validation_0-f1_err:0.955898
[11]	validation_0-error:0.050595	validation_0-f1_err:0.954912
[12]	validation_0-error:0.046239	validation_0-f1_err:0.958744
[13]	validation_0-error:0.048417	validation_0-f1_err:0.956911
[14]	validation_0-error:0.045317	validation_0-f1_err:0.95957
[15]	validation_0-error:0.045988	validation_0-f1_err:0.958978
[16]	validation_0-er

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=1,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

Fit on the whole dataset

In [79]:
xgb_model=xgb.XGBClassifier(learning_rate= 0.1, max_depth= 1, n_estimators= 500)

xgb_model.fit(X=pred_df, y=binlabels)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=1,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

# Make predictions on test set

In [80]:
subpreds=xgb_model.predict(testpred_df)

In [81]:
np.mean(subpreds)

0.5727205734574932

# Prep submission

In [82]:
sampledf=pd.read_csv(f'{mount_path}{competition_path}sample_submission.csv')
imglst=['/content/All_Data/'+ii+'.jpg' for ii in sampledf['id']]
imgnames=[os.path.basename(img)[:-4] for img in imglst]

### Make submission file

In [83]:
labeled_preds=subpreds.copy().astype(str)

# subpreds=np.argmin(curpreds,axis=1)

labeled_preds[subpreds==0]='venomous' 
labeled_preds[subpreds==1]='non_venomous'
print(labeled_preds)

['non_venomous' 'non_venomous' 'venomous' ... 'venomous' 'venomous'
 'non_venomous']


In [84]:
submitdf=pd.DataFrame(data=np.stack([imgnames,labeled_preds],1),columns=['id','class'])

submitdf.head()

,id,class
0,00022e97c90898c02a25579be,non_venomous
1,000816b96db153cc47dfb79db,non_venomous
2,001d2fe5ca243c138472d130b,venomous
3,001e3329abba16fb91ee4d47f,non_venomous
4,0026cf1ed9556341340419b19,venomous


In [86]:
datestr=datetime.datetime.now(pytz.timezone('US/Eastern')).strftime("%y%m%d_%H%M")
submitdf.to_csv('/content/'+datestr+'_submission.csv',index=False)